### Import Required Libraries and Set Up Environment Variables

In [185]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [147]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [148]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [149]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page_num in range(20):

    # create query with a page number
    query_url = f"{query_url}&page="
    
    # API results show 10 articles at a time (results are limited to 10 per page)
    page_url = query_url + str(page_num)
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_url)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        for review in reviews["response"]["docs"]:

        # loop through the reviews["response"]["docs"] and append each review to the list
            reviews_list.append(review)
            
        # Print the page that was just retrieved
        print(f"Retrieved reviews from page: {page_num}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"No results found on page: {page_num}.  Breaking from the loop.")
        break

Retrieved reviews from page: 0
Retrieved reviews from page: 1
Retrieved reviews from page: 2
Retrieved reviews from page: 3
Retrieved reviews from page: 4
Retrieved reviews from page: 5
Retrieved reviews from page: 6
Retrieved reviews from page: 7
Retrieved reviews from page: 8
Retrieved reviews from page: 9
Retrieved reviews from page: 10
Retrieved reviews from page: 11
Retrieved reviews from page: 12
Retrieved reviews from page: 13
Retrieved reviews from page: 14
Retrieved reviews from page: 15
Retrieved reviews from page: 16
Retrieved reviews from page: 17
Retrieved reviews from page: 18
Retrieved reviews from page: 19


In [169]:
# Preview the first 5 results in JSON format
first_five_results = reviews_list[:5]

# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews, indent=4))


{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [190]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_df_normalized = pd.json_normalize(reviews_list)
reviews_df_normalized['title'] = reviews_df_normalized['headline.main'].str.extract(r'\u2018(.*?)\u2019.*?Review')
print(reviews_df_normalized)

                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2023/04/13/movies/one-...   
196  https://www.nytimes.com/2023/04/13/movies/the-...   
197  https://www.nytimes.com/2023/03/30/movies/a-th...   
198  https://www.nytimes.com/2023/02/09/movies/your...   
199  https://www.nytimes.com/2023/02/02/movies/love...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Religion comes between two girls falling in lo...  The New York 

In [200]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df_normalized['keywords'] = reviews_df_normalized['keywords'].apply(lambda x: ', '.join(x))


In [201]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df_normalized['title'].to_list()

### Access The Movie Database API

In [154]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [203]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print("Application is sleeping...")
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    search_url = "https://api.themoviedb.org/3/search/movie"
    params = {
        "query": title,
        "api_key": tmdb_api_key
    }
    response = requests.get(search_url, params=params)
    search_results = response.json()

    # Include a try clause to search for the full movie details.
    try:
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id
        movie_id = search_results["results"][0]["id"]

        # Make a request for a the full movie details
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}"

        # Execute "GET" request with url
        movie_response = requests.get(movie_url, params={"api_key": tmdb_api_key})
        
        movie_details = movie_response.json()
        
        # Extract the genre names into a list
        genres = [genre["name"] for genre in movie_details["genres"]]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang["english_name"] for lang in movie_details["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in movie_details["production_countries"]]

        # Add the relevant data to a dictionary and
        movie_dict = {
            "title": movie_details["title"],
            "original_title": movie_details["original_title"],
            "budget": movie_details["budget"],
            "original_language": movie_details["original_language"],
            "homepage": movie_details["homepage"],
            "overview": movie_details["overview"],
            "popularity": movie_details["popularity"],
            "runtime": movie_details["runtime"],
            "revenue": movie_details["revenue"],
            "release_date": movie_details["release_date"],
            "vote_average": movie_details["vote_average"],
            "vote_count": movie_details["vote_count"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        }

        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(movie_dict)
        
        # Print out the title that was found
        print(f"Title found: {movie_details['title']}")

    except IndexError:
        print(f"Movie not found for title: {title}")


Title found: The Attachment Diaries
Title found: Bread
Title found: You Can Live Forever
Title found: A Tourist's Guide to Love
Title found: Other People
Title found: One True Loves
Title found: The Lost Weekend: A Love Story
Title found: A Thousand and One
Title found: Your Place or Mine
Title found: Love in the Time of Fentanyl
Title found: The Attachment Diaries
Title found: Bread
Title found: You Can Live Forever
Title found: A Tourist's Guide to Love
Title found: Other People
Title found: One True Loves
Title found: The Lost Weekend: A Love Story
Title found: A Thousand and One
Title found: Your Place or Mine
Title found: Love in the Time of Fentanyl
Title found: The Attachment Diaries
Title found: Bread
Title found: You Can Live Forever
Title found: A Tourist's Guide to Love
Title found: Other People
Title found: One True Loves
Title found: The Lost Weekend: A Love Story
Title found: A Thousand and One
Title found: Your Place or Mine
Title found: Love in the Time of Fentanyl
Titl

In [204]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 5.486,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [206]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [207]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df_normalized, tmdb_df, on='title', how='inner')

In [214]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:        
        
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df.head)

<bound method NDFrame.head of                                              web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   
2  https://www.nytimes.com/2023/04/20/movies/othe...   
3  https://www.nytimes.com/2023/04/13/movies/one-...   
4  https://www.nytimes.com/2023/04/13/movies/the-...   
5  https://www.nytimes.com/2023/03/30/movies/a-th...   
6  https://www.nytimes.com/2023/02/09/movies/your...   
7  https://www.nytimes.com/2023/02/02/movies/love...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   
2  A radiant Virginie Efira stars as a Parisian t...  The New York Times   
3  A film adaptation of Taylor Jenkins Reid’s nov...  The New York Times   
4  There’s not much Lennon music heard in this do...  The New York Times   
5  A mesm

In [211]:
# Drop "byline.person" column
merged_df = merged_df.drop("byline.person", axis=1)

In [212]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.reset_index(drop=True)

In [213]:
# Export data to CSV without the index
merged_df.to_csv("merged_data.csv", index=False)